## Station and pollutant selection

Stations provide hourly measurements.

For a day to be considered valid **for a given station and pollutant**, it must have at least 75% of observations, i.e., at least 18 observations.

For a year to be considered valid **for a given station and pollutant**, it must have at least 75% of valid days, i.e., at least 328 days.

For a station–pollutant pair to be considered in subsequent analyses, it must have at least two valid years.

## Initialization

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Data Directory
dir_files = './03-dados-qualar-longo-corrigido/'

In [ ]:
# Measures File
ficheiro_medicoes = dir_files + '03-medicoes-longo.csv'

In [ ]:
medicoes = pd.read_csv(ficheiro_medicoes, thousands=',', index_col=0, parse_dates=True)
medicoes

In [ ]:
# Create a column with only the day from the date
medicoes['Dia'] =medicoes.index.date
# Create a column with only the year from the date
medicoes['Ano'] = medicoes.index.year
# Check result
medicoes

## Calculation of hours per day and valid days
We will analyze the number of valid days present in the data.

In [ ]:
lista_estacoes = medicoes['Estação'].unique()
lista_estacoes

In [ ]:
lista_poluentes = medicoes['variable'].unique()
lista_poluentes

In [ ]:
# Calculate hours per day
validade_diaria = medicoes.groupby(['Estação', 'variable', 'Dia']).size().reset_index(name = 'Horas')

In [ ]:
validade_diaria

In [ ]:
# Calculate valid days
validade_diaria['Validade'] = validade_diaria['Horas'].apply(lambda x: 'Válido' if x>=18 else 'Inválido')
validade_diaria

In [ ]:
# Show valid and invalid days in the data
dias_validos = validade_diaria['Validade'].value_counts()
dias_validos

## Calculation of valid days per year and valid years
We will analyze the number of valid years present in the data.

In [ ]:
# Prepare the DataFrame with valid-day and year information
dias_validados = validade_diaria.loc[validade_diaria['Validade'] == 'Válido']
dias_validados = dias_validados.copy()
dias_validados['Ano'] = dias_validados['Dia'].apply(lambda x: x.year)
dias_validados

In [ ]:
# Calculate valid days per year
validade_anual = dias_validados.groupby(['Estação', 'variable', 'Ano']).size().reset_index(name = 'Dias')
validade_anual

In [ ]:
# Calculate valid years per station and pollutant
validade_anual['Validade'] = validade_anual['Dias'].apply(lambda x: 'Válido' if x>=328 else 'Inválido')
validade_anual

In [ ]:
# Show valid and invalid years in the data
anos_validos = validade_anual['Validade'].value_counts()
anos_validos

## Valid pollutant and station measurements
We will keep only measurements corresponding to valid years.

In [ ]:
# Merge valid-year information into the measurements
anos_validados = medicoes.merge(validade_anual[['Estação', 'variable', 'Ano', 'Validade']], on=['Estação', 'variable', 'Ano'], how='inner')
anos_validados

In [ ]:
# Keep only measures of valid years
anos_validados = anos_validados.loc[anos_validados['Validade'] == 'Válido']
anos_validados

In [ ]:
# Calculate valid measures per station
med_validas_estacao = anos_validados.groupby(['Estação']).size()
med_validas_estacao

In [ ]:
# Calculate valid measures per pollutant
med_validas_poluente = anos_validados.groupby(['variable']).size()
med_validas_poluente